In [ ]:
from pr2_utils import *
import gtsam
from gtsam.symbol_shorthand import P  # Pose variable
from gtsam import NonlinearFactorGraph, Values, Pose2, BetweenFactorPose2, Rot2

In [ ]:
dataset = 20
odo_str = "icp"

with np.load(f"../data/odometry_icp_{dataset}.npz") as data:
    icp_odometry = data["X"]
    icp_odometry_stamp = data["stamps"]

with np.load(f"../data/odometry_imu_{dataset}.npz") as data:
    imu_odometry = data["X"]
    imu_odometry_stamp = data["stamps"]

# using icp stamp as key
f_imu_odometry = InterpN1D(imu_odometry_stamp, imu_odometry)
f_icp_odometry = InterpN1D(icp_odometry_stamp, icp_odometry)

In [ ]:
# graph = NonlinearFactorGraph()

# initial_estimate = Values()
# for i, t in enumerate(imu_odometry_stamp):
#     initial_estimate.insert(P(i), Pose2(*icp_odometry[i]))

# # Add a prior for the first pose, assuming we start at the origin with no uncertainty
# graph.add(gtsam.PriorFactorPose2(P(0), Pose2(0, 0, 0), gtsam.noiseModel.Diagonal.Sigmas([0.1, 0.1, 0.1])))
# icp_noise = gtsam.noiseModel.Diagonal.Sigmas([1, 1, 1])
# delta_icp_odometry = icp_odometry[1:] - icp_odometry[:-1]
# for i, icp_delta_odo in enumerate(delta_icp_odometry):
#     delta_pose = Pose2(*icp_delta_odo)
#     graph.add(BetweenFactorPose2(P(i), P(i+1), delta_pose, icp_noise))

In [ ]:
graph = NonlinearFactorGraph()

time_stamp = icp_odometry_stamp
icp_odometry_t = f_icp_odometry(time_stamp)
imu_odometry_t = f_imu_odometry(time_stamp)

initial_estimate = Values()
for i, t in enumerate(time_stamp):
    initial_estimate.insert(P(i), Pose2(*imu_odometry_t[i]))

# Add a prior for the first pose, assuming we start at the origin with no uncertainty
graph.add(gtsam.PriorFactorPose2(P(0), Pose2(0, 0, 0), gtsam.noiseModel.Diagonal.Sigmas([0, 0, 0])))
icp_noise = gtsam.noiseModel.Diagonal.Sigmas([5, 5, 1])
delta_icp_odometry = icp_odometry_t[1:] - icp_odometry_t[:-1]
for i, icp_delta_odo in enumerate(delta_icp_odometry):
    delta_pose = Pose2(*icp_delta_odo)
    graph.add(BetweenFactorPose2(P(i), P(i+1), delta_pose, icp_noise))

# imu_noise = gtsam.noiseModel.Diagonal.Sigmas([0.05, 0.05, 0.01])
# delta_imu_odometry = f_imu_odometry(icp_odometry_stamp[1:]) - f_imu_odometry(icp_odometry_stamp[:-1])
# for i, delta_imu_odo in enumerate(delta_imu_odometry):
#     delta_pose = Pose2(*delta_imu_odo)
#     graph.add(BetweenFactorPose2(P(i), P(i+1), delta_pose, imu_noise))

In [ ]:
params = gtsam.LevenbergMarquardtParams()
params.setVerbosity("Termination")  # this will show info about stopping conds
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()

In [ ]:
factor_graph_optimized = []
for i in range(icp_odometry_stamp.shape[0]):
    pose = result.atPose2(P(i))
    factor_graph_optimized.append([pose.x(), pose.y(), pose.theta()])
factor_graph_optimized = np.array(factor_graph_optimized)

plot_odometry([
    (imu_odometry, imu_odometry_stamp, "imu (motion)"),
    (icp_odometry, icp_odometry_stamp, "icp (observation)"),
    (factor_graph_optimized, icp_odometry_stamp, "factor_graph_optimized")
])

In [ ]:
icp_odometry_stamp.shape[0]

In [ ]:
type(result.atPose2(P(1)))